In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Lung_Cancer"
cohort = "GSE244117"

# Input paths
in_trait_dir = "../../input/GEO/Lung_Cancer"
in_cohort_dir = "../../input/GEO/Lung_Cancer/GSE244117"

# Output paths
out_data_file = "../../output/preprocess/Lung_Cancer/GSE244117.csv"
out_gene_data_file = "../../output/preprocess/Lung_Cancer/gene_data/GSE244117.csv"
out_clinical_data_file = "../../output/preprocess/Lung_Cancer/clinical_data/GSE244117.csv"
json_path = "../../output/preprocess/Lung_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# This dataset appears to be spatial transcriptomics data of olfactory neuroblastoma (ONB) samples
# The description mentions transcriptomic profiling with the NanoString GeoMx platform
# It's not miRNA or methylation data, but rather whole transcriptome analysis
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the Sample Characteristics Dictionary

# 2.1 Data Availability
# For trait:
# Row 1 contains 'grade' information which can be used as our trait
# Grade indicates cancer severity/progression
trait_row = 1

# For age:
# Row 5 contains age information
age_row = 5

# For gender:
# Row 4 contains gender information as 'Sex: F' or 'Sex: M'
gender_row = 4

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert grade information to binary cancer status."""
    if pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (normal=0, any grade=1)
    if value.lower() == 'normal':
        return 0
    elif value.upper() in ['I', 'II', 'III', 'IV']:
        return 1
    else:
        return None

def convert_age(value):
    """Convert age string to numerical value."""
    if pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender string to binary (F=0, M=1)."""
    if pd.isna(value):
        return None
    
    # Extract the value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Skip this step if trait_row is None
if trait_row is not None:
    try:
        # Extract clinical features using the clinical_data variable that was mentioned in the instructions
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Save the extracted clinical features to a CSV file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    except NameError:
        print("The 'clinical_data' variable was not found. Please make sure it's properly defined.")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Analyzing the gene identifiers shown in the output
# These identifiers appear to be standard human gene symbols (HGNC symbols)
# For example: A1BG (alpha-1-B glycoprotein), A2M (alpha-2-macroglobulin), AAAS (aladin WD repeat nucleoporin)
# These are the standard official gene symbols for human genes, not probe IDs or other identifiers

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
# Save to the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
# Need to load the saved clinical data - it's saved as T, so we need to transpose it back
clinical_features = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Shape of clinical_features before transpose: {clinical_features.shape}")
print(f"Columns in clinical_features: {clinical_features.columns.tolist()[:5]}...")  # Show first 5 columns

# Convert the clinical dataframe to the expected format for linking
# The index should be the feature names (Lung_Cancer, Age, Gender) and the columns should be sample IDs
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")
print(f"First few columns in linked_data: {linked_data.columns.tolist()[:5]}")

# 3. Handle missing values in the linked data
# First check column names to ensure we're using the right trait name
print(f"Available columns in linked_data: {[col for col in linked_data.columns if col in [trait, 'Age', 'Gender', '0', '1']]}")

# The trait name in the linked data might be "0" (first row index from clinical_features)
# We need to use the actual column name that exists in the linked data
trait_col = "0"  # This assumes the first row of clinical_features had the trait data
linked_data = handle_missing_values(linked_data, trait_col)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# Rename columns to ensure consistent naming
if trait_col != trait:
    linked_data = linked_data.rename(columns={trait_col: trait})

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from olfactory neuroblastoma patients relevant to lung cancer research"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Data not saved.")


### Step 6: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")
# Save to the gene data file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Link the clinical and genetic data
# Need to load the saved clinical data
clinical_features = pd.read_csv(out_clinical_data_file)

# Print clinical_features structure to debug
print(f"Clinical features columns: {clinical_features.columns.tolist()}")
print(f"Clinical features shape: {clinical_features.shape}")

# Convert clinical features to the correct format for linking
# First, we need to set the appropriate index
if 'Unnamed: 0' in clinical_features.columns:
    clinical_features = clinical_features.set_index('Unnamed: 0')

# Create a new DataFrame with the trait name as Lung_Cancer for clarity
trait_row = clinical_features.iloc[0].rename(trait)
age_row = clinical_features.iloc[1].rename('Age') if len(clinical_features) > 1 else None
gender_row = clinical_features.iloc[2].rename('Gender') if len(clinical_features) > 2 else None

# Combine rows into a new clinical dataframe with proper naming
clinical_df_rows = [trait_row]
if age_row is not None:
    clinical_df_rows.append(age_row)
if gender_row is not None:
    clinical_df_rows.append(gender_row)

named_clinical_df = pd.DataFrame(clinical_df_rows)
print(f"Named clinical dataframe shape: {named_clinical_df.shape}")
print(f"Named clinical dataframe index: {named_clinical_df.index.tolist()}")

# Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(named_clinical_df, normalized_gene_data)
print(f"Shape of linked data: {linked_data.shape}")
print(f"First few columns in linked_data: {linked_data.columns[:10].tolist()}")

# Check if the trait column exists in the dataframe
if trait not in linked_data.columns:
    print(f"Warning: '{trait}' column not found in linked data. Available columns: {linked_data.columns[:20].tolist()}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are biased, and remove biased features
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from olfactory neuroblastoma patients relevant to lung cancer research"
)

# 6. If the linked data is usable, save it as a CSV file
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed. Data not saved.")